In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from random import sample
import catboost
from sklearn import preprocessing

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
sample_submission = pd.read_csv("/kaggle/input/lish-moa/sample_submission.csv")
train_targets_scored =  pd.read_csv("/kaggle/input/lish-moa/train_targets_scored.csv")
train_targets_nonscored = pd.read_csv("/kaggle/input/lish-moa/train_targets_nonscored.csv")
train_features= pd.read_csv("/kaggle/input/lish-moa/train_features.csv")
test_features =  pd.read_csv("/kaggle/input/lish-moa/test_features.csv")

# wanted to iterate on data sets to print their heads, did not work
# all_data_sets = {"sample_submission": sample_submission, "train_targets_scored": train_targets_scored,
#                  "train_targets_nonscored": train_targets_nonscored, "train_features": train_features,
#                  "test_features": test_features}

In [ ]:
le_cp_type = preprocessing.LabelEncoder()
le_cp_dose = preprocessing.LabelEncoder()

train_features["cp_type"] = le_cp_type.fit_transform(train_features["cp_type"])
train_features["cp_dose"] = le_cp_dose.fit_transform(train_features["cp_dose"])

test_features["cp_type"] = le_cp_type.transform(test_features["cp_type"])
test_features["cp_dose"] =le_cp_dose.transform(test_features["cp_dose"])

In [ ]:
train_targets_scored.shape

In [ ]:
def train_all(X, y):
    models_dict = {}
    print("training")
    for i, col in enumerate(y.columns):
        if col != "sig_id":
            print(i, end = ", ")
            clf = catboost.CatBoostClassifier(depth=3)
            clf.fit(X.drop("sig_id", axis = 1), y[col])
            models_dict[col] = clf
    return models_dict
            
models_dict = train_all(train_features, train_targets_scored)

print("done training")

print("predicting")
for medicen in train_targets_scored.drop("sig_id", axis = 1).columns:
    sample_submission[medicen] = pd.Series([pred[1] for pred in 
                                            models_dict[medicen].predict_proba(test_features.drop("sig_id", axis = 1))])
    

print("done predicting")

In [ ]:
sample_submission.to_csv("submission.csv", index = False)